In [1]:
cd ..

/Users/yu-lin.huang/git/statec/statnews


In [2]:
import ollama

In [3]:
from src.utils.env import INDICATOR_URL, SHEET_ANNUAL_VARIATION, SHEET_MENSUEL_VARIATION
from src.preprocess.ingestion import get_sheet, get_top_variations, get_target_index

In [4]:
def construire_prompt(target, variation, annual_variation, positives, negatives):
    return f"""
Tu es un assistant économique chargé d’écrire une courte analyse des prix alimentaires mensuels.

Le mois cible est : {target}

📊 Variation mensuelle : {variation}%  
📊 Variation annuelle : {annual_variation}%  
🔺 Produits en hausse : {positives}  
🔻 Produits en baisse : {negatives}

Rédige un paragraphe journalistique de 3 à 5 phrases.

Consignes :
- Commence par annoncer la variation globale (hausse ou baisse) en mensuelle.
- Mentionne le produit dont les prix a plus baissé (minimum) et deux autre produits dont les prix ont le plus baissé
- Mentionne le produit dont les prix a plus augmenté (maximum) et deux autre dont les prix ont le plus augmenté.
- Intègre les pourcentages entre parenthèses, avec - si négative.
- Utilise un ton neutre, fluide et informatif.
- Décrire juste la variation sans donner de commentaire ou conclusion.
- Terminer par annoncer la variation globale (hausse ou baisse) en annuel.
"""


In [5]:
df_mensuel = get_sheet(SHEET_MENSUEL_VARIATION)
df_annual = get_sheet(SHEET_ANNUAL_VARIATION)

In [6]:
target = 2024.06
alimentaire_mensuel_negative = get_top_variations(df_mensuel, target=target, prefix="01", positive=False, level='4', n=10)
alimentaire_mensuel_positive = get_top_variations(df_mensuel, target=target, prefix="01", positive=True, level='4', n=10)
alimentaire_mensuel_variation = get_target_index(df_mensuel, target=target, code='0101')
alimentaire_annuel_variation = get_target_index(df_annual, target=target, code='0101')

In [7]:
alimentaire_mensuel_negative

{'Nourriture pour bébés': -3.9,
 'Fruits frais ': -2.6,
 'Poisson surgelé': -2.3,
 'Fruits de mer surgelés': -1.9,
 'Légumes surgelés': -1.9,
 'Fruits surgelés': -1.8,
 'Oeufs': -1.8,
 'Céréales pour petit déjeuner': -1.7,
 'Autres graisses animales': -1.6,
 'Autres huiles alimentaires': -1.2}

In [8]:
prompt = construire_prompt(target, alimentaire_mensuel_variation, alimentaire_annuel_variation, alimentaire_mensuel_positive, alimentaire_mensuel_negative)

In [9]:
response = ollama.generate(
    model="qwen3:8b",
    prompt=prompt,
)

In [10]:
print(response.response)

En juin 2024, les prix alimentaires ont légèrement baissé de 0,4 % par rapport au mois précédent. Les produits les plus en baisse incluent la nourriture pour bébés (-3,9 %), suivis des fruits frais (-2,6 %) et du poisson surgelé (-2,3 %). En revanche, les prix ont augmenté pour l’huile d’olive (4,1 %), les fruits de mer frais (3,2 %) et les chips (3,2 %). La variation annuelle des prix alimentaires s’élève à une hausse de 1,6 % sur l’année.


In [11]:
alimentaire_mensuel_negative

{'Nourriture pour bébés': -3.9,
 'Fruits frais ': -2.6,
 'Poisson surgelé': -2.3,
 'Fruits de mer surgelés': -1.9,
 'Légumes surgelés': -1.9,
 'Fruits surgelés': -1.8,
 'Oeufs': -1.8,
 'Céréales pour petit déjeuner': -1.7,
 'Autres graisses animales': -1.6,
 'Autres huiles alimentaires': -1.2}

In [12]:
import requests

In [ ]:
token = ""

In [16]:

def chat_with_model(token, prompt):
    url = 'http://10.6.35.12:8080/ollama/api/generate'
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    data = {
      "model": "mistral:7b",
      "prompt": prompt,
      "stream": False
    }
    response = requests.post(url, headers=headers, json=data)
    return response

In [17]:
response = chat_with_model(token, prompt)

In [18]:
print(response.json()['response'])

 Les prix des produits alimentaires ont baissé en moyenne de -0.4% en juin 2024. Parmi les produits, les fruits frais ont connu la plus forte baisse avec -2,6%, suivis par le poisson surgelé (-2,3%) et les légumes surgelés (-1,9%). En revanche, les prix des huiles alimentaires, notamment l'huile d'olive (+4,1%), ont augmenté en moyenne de 4,1%. Autres produits marqués par une augmentation des prix : les fruits de mer frais avec +3,2%, les chips et les boissons gazeuses non minérales (+1,3%) et le poisson et fruits de mer séchés, fumés ou salés (+1%). Globalement, les prix ont augmenté en moyenne de 1,6% au cours de l'année 2024.
